# Multi-order Healpix

In [75]:
# imports
import numpy as np

import healpy 

from astropy import units
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
from astropy.table import Table
from astropy.io import fits

import astropy_healpix

from matplotlib import pyplot as plt

from ligo.skymap.io.fits import write_sky_map
import reproject
import tempfile
#from mocpy import MOC

# Set nside

In [2]:
nside = astropy_healpix.pixel_resolution_to_nside(0.1*units.arcsec)
nside

2097152

In [17]:
level = int(np.log2(nside))
level

21

## This seems one number to low..

# FRB localization centroid (fake)

In [4]:
frb_cen = SkyCoord(ra=326.1052292, dec=-40.9000278, unit='deg')

In [7]:
pa_FRB = Angle(45, units.deg)
a_FRB = Angle(0.4, units.arcsec)
b_FRB = Angle(0.3, units.arcsec)

# Find pixel indices inside the localization

    https://zonca.dev/2020/10/example-healpy-query_disc.html

In [26]:
lon_FRB = frb_cen.galactic.l.deg
lat_FRB = frb_cen.galactic.b.deg
vec = healpy.ang2vec(lon_FRB, lat_FRB, lonlat=True)

In [43]:
lon_FRB, lat_FRB

(0.7424470849499389, -49.414664083389745)

## Grab the pixels

In [20]:
ipix = healpy.query_disc(nside, vec, radius=3*a_FRB.to('rad').value)

In [22]:
len(ipix)

448

## UNIQ

In [25]:
uniq = astropy_healpix.level_ipix_to_uniq(level, ipix)
uniq[0:10]

array([64020621899339, 64020621899340, 64020621899341, 64020621899342,
       64020629025693, 64020629025694, 64020629025695, 64020629025696,
       64020629025697, 64020629025698], dtype=int64)

## Healpix coordinates

In [50]:
lon_pix, lat_pix = astropy_healpix.healpix_to_lonlat(ipix, nside)#, order='nested')

In [51]:
frb_cen.galactic.l.to('rad'), frb_cen.galactic.b.to('rad')

(<Longitude 0.01295815 rad>, <Latitude -0.86244859 rad>)

In [52]:
lon_pix[0], lat_pix[0]

(<Longitude 0.01295677 rad>, <Latitude -0.86244292 rad>)

In [54]:
heal_coord = SkyCoord(lon_pix, lat_pix, frame='galactic')

In [55]:
heal_coord[0]

<SkyCoord (Galactic): (l, b) in deg
    (0.74236819, -49.41433935)>

### Minimum separation

In [31]:
np.min(heal_coord[0].separation(heal_coord[1:])).to('arcsec')

<Angle 0.11734893 arcsec>

## PDF

In [56]:
sep = frb_cen.separation(heal_coord)
np.min(sep).to('arcsec')

<Angle 0.06054927 arcsec>

In [57]:
pa_healpy = frb_cen.position_angle(heal_coord)

In [58]:
dtheta = 90.*units.deg - pa_FRB  # Place a of ellipse along the x-axis

In [59]:
new_pa_healpy = pa_healpy + dtheta

In [60]:
x_hp = -sep * np.sin(new_pa_healpy)
y_hp = sep * np.cos(new_pa_healpy)
p_xy = np.exp(-x_hp**2 / (2*a_FRB**2)) * np.exp(-y_hp**2 / (2*b_FRB**2))

In [63]:
np.max(p_xy)

<Quantity 0.98168845>

## Write

In [68]:
tbl = Table()
tbl['UNIQ'] = uniq
tbl['PROBDENSITY'] = p_xy

In [78]:
name = 'tmp.fits'
write_sky_map(name, tbl,
              vcs_version='foo 1.0', vcs_revision='bar',
              build_date='2018-01-01T00:00:00')
for card in fits.getheader(name, 1).cards:
    print(str(card).rstrip())

XTENSION= 'BINTABLE'           / binary table extension
BITPIX  =                    8 / array data type
NAXIS   =                    2 / number of array dimensions
NAXIS1  =                   16 / length of dimension 1
NAXIS2  =                  448 / length of dimension 2
PCOUNT  =                    0 / number of group parameters
GCOUNT  =                    1 / number of groups
TFIELDS =                    2 / number of table fields
TTYPE1  = 'UNIQ    '
TFORM1  = 'K       '
TTYPE2  = 'PROBDENSITY'
TFORM2  = 'D       '
TUNIT2  = ''
PIXTYPE = 'HEALPIX '           / HEALPIX pixelisation
ORDERING= 'NUNIQ   '           / Pixel ordering scheme: RING, NESTED, or NUNIQ
COORDSYS= 'C       '           / Ecliptic, Galactic or Celestial (equatorial)
MOCORDER=                   21 / MOC resolution (best order)
INDXSCHM= 'EXPLICIT'           / Indexing: IMPLICIT or EXPLICIT
VCSVERS = 'foo 1.0 '           / Software version
VCSREV  = 'bar     '           / Software revision (Git)
DATE-BLD= '2018-

## Plot

    https://reproject.readthedocs.io/en/stable/healpix.html

In [76]:
target_header = fits.Header.fromstring("""
NAXIS   =                    2
NAXIS1  =                  480
NAXIS2  =                  240
CTYPE1  = 'RA---MOL'
CRPIX1  =                240.5
CRVAL1  =                180.0
CDELT1  =               -0.675
CUNIT1  = 'deg     '
CTYPE2  = 'DEC--MOL'
CRPIX2  =                120.5
CRVAL2  =                  0.0
CDELT2  =                0.675
CUNIT2  = 'deg     '
COORDSYS= 'icrs    '
""", sep='\n')

In [80]:
hdul = fits.open('tmp.fits')
hdul.info()

Filename: tmp.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     21   448R x 2C   [K, D]   


In [82]:
data = Table(hdul[1].data)
data

UNIQ,PROBDENSITY
int64,float64
64020621899339,0.005033734428770343
64020621899340,0.004136156745120631
64020621899341,0.002992680390782034
64020621899342,0.0019066912752544278
64020629025693,0.010380793719408227
64020629025694,0.0112592926772825
64020629025695,0.010753460048620322
64020629025696,0.009043612446175584
64020629025697,0.006697184886557686


In [84]:
array, footprint = reproject.reproject_from_healpix('tmp.fits',
                                          target_header)#, nested=True)

ValueError: Number of pixels must be divisible by 12

----

## MOC --  deprecated

In [6]:
frb_cen.galactic.b

<Latitude -49.41466408 deg>

In [7]:
Angle(frb_cen.galactic.l), Angle(frb_cen.galactic.b)

(<Angle 0.74244708 deg>, <Angle -49.41466408 deg>)

In [54]:
moc = MOC.from_elliptical_cone(lon=Angle(frb_cen.galactic.l), 
                               lat=Angle(frb_cen.galactic.b),
                                a=3*a_FRB, b=3*b_FRB, 
                               pa=pa_FRB,
                               max_depth=max_order+1) # Note the +1

In [55]:
uniq = moc._uniq_format()
uniq

array([   868166242011,    868166242032,   3472664968038,   3472664968039,
         3472664968041,   3472664968043,   3472664968132,   3472664968134,
        13890659872141,  13890659872143,  13890659872147,  13890659872150,
        13890659872151,  13890659872163,  13890659872169,  13890659872171,
        13890659872202,  13890659872224,  13890659872226,  13890659872532,
        13890659872533,  13890659872534,  13890659872540,  13890659872545,
        13890659872548,  13890659872549,  13890659872550,  13890659872560,
        55562639488569,  55562639488570,  55562639488571,  55562639488583,
        55562639488585,  55562639488586,  55562639488587,  55562639488594,
        55562639488595,  55562639488645,  55562639488647,  55562639488673,
        55562639488675,  55562639488681,  55562639488682,  55562639488683,
        55562639488778,  55562639488800,  55562639488802,  55562639488803,
        55562639488928,  55562639488929,  55562639488930,  55562639488936,
        55562639488938,  

In [56]:
type(uniq)

numpy.ndarray

In [57]:
moc.spatial_resolution_to_order(Angle(0.1, units.arcsec))

22

# Grab RA, DEC

In [77]:
level, ipix = astropy_healpix.uniq_to_level_ipix(uniq)
level, ipix

(array([18, 18, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 20,
        20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21,
        21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22,
        22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
        22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
        22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
        22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
        22, 22, 22, 22, 22]),
 array([   593288335067,    593288335088,   2373153340262,   2373153340263,
          2373153340265,   2373153340267,   2373153340356,   2373153340358,
          9492613361037,   9492613361039,   9492613361043,   9492613361046,
          9492613361047,   9492613361059,   9492613361065,   9492613361067,
          9492613361098,   9492613361120,   9492613361122,

## New nside

In [59]:
new_nside = astropy_healpix.level_to_nside(level)
#new_nside

## lon, lat

In [61]:
lon, lat = astropy_healpix.healpix_to_lonlat(ipix, new_nside, order='nested')
#lon.to('deg'), lat.to('deg')

# PDF

In [62]:
heal_coord = SkyCoord(lon, lat, frame='galactic')
#heal_coord.icrs.ra, heal_coord.icrs.dec

In [63]:
frb_cen

<SkyCoord (ICRS): (ra, dec) in deg
    (326.1052292, -40.9000278)>

In [65]:
sep = frb_cen.separation(heal_coord)
#sep.to('arcsec')

## Ellipse

In [66]:
pa_healpy = frb_cen.position_angle(heal_coord)

In [67]:
dtheta = 90.*units.deg - pa_FRB  # Place a of ellipse along the x-axis

In [68]:
new_pa_healpy = pa_healpy + dtheta

In [69]:
x_hp = -sep * np.sin(new_pa_healpy)
y_hp = sep * np.cos(new_pa_healpy)

In [72]:
p_xy = np.exp(-x_hp**2 / (2*a_FRB**2)) * np.exp(-y_hp**2 / (2*b_FRB**2))

In [73]:
p_xy

<Quantity [5.21415321e-01, 2.24118955e-01, 2.19358156e-01, 2.08587279e-01,
           6.83937451e-02, 3.26362053e-01, 4.33241358e-01, 7.90175867e-02,
           9.71218615e-03, 3.70009170e-02, 8.59151599e-02, 1.38259489e-01,
           8.72887660e-02, 2.08184814e-03, 7.50499523e-03, 2.12598685e-02,
           1.41564587e-02, 7.97929220e-03, 3.53413115e-03, 4.95978946e-02,
           3.02312027e-03, 1.63340131e-02, 4.22698746e-03, 1.04289678e-01,
           1.79839068e-01, 1.21664395e-01, 8.58826921e-02, 3.22909394e-02,
           3.90820453e-03, 1.81703033e-03, 8.14033088e-03, 4.84933883e-02,
           2.34420496e-02, 1.81283078e-02, 4.44950964e-02, 6.97243752e-02,
           7.93407306e-02, 7.43879860e-04, 1.62308727e-03, 4.18702281e-04,
           8.48346447e-04, 1.61833477e-03, 4.96392313e-04, 2.90663620e-03,
           3.41554043e-02, 3.40240075e-02, 3.19108718e-02, 1.02011947e-02,
           3.76852406e-03, 6.71531612e-04, 2.18243653e-03, 1.18997836e-03,
           6.10890842e-04

In [74]:
np.argmax(p_xy), np.argmin(p_xy)

(0, 138)

In [76]:
sep[0].to('arcsec')

<Angle 0.36464952 arcsec>